## Part 1: prep data

In [145]:
# import matlab data
import numpy as np
import scipy.io
from random import sample
mat =scipy.io.loadmat('training_set_5_psth.mat')
input_training = mat['input_training']
output_training = mat['output_training']

In [166]:
# divide into training and validation sets
nullSamples = output_training < 0.001;
X = input_training[nullSamples.ravel()==False];
Y = output_training[nullSamples.ravel()==False];

trainBatch = 0.9;
m = X.shape[0]
m_train = sample(range(m),round(m*trainBatch))
Xtrain = X[m_train,:]
Ytrain = Y[m_train,:]
Xtest = X.copy()
Xtest = np.delete(Xtest,m_train,axis=0)
Ytest = Y.copy()
Ytest = np.delete(Ytest,m_train,axis=0)

In [169]:
# make sure that X.shape[0] = m = Xtrain.shape[0]+Xtest.shape[0]
print('Checking whether number of samples add up...')
print(m == Xtrain.shape[0]+Xtest.shape[0])


Checking whether number of samples add up...
True


## Part 2: The Network

In [170]:
from keras import layers
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

### Fully Connected Inhibitory Input Layer
our network requires a custom layer to represent inhibition from one class of neurons to another class of neurons.

The synaptic weights should only take on positive values

In [179]:
import tensorflow as tf
class FCLayer_InhibitoryInput(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(FCLayer_InhibitoryInput, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    W = np.log(1+np.exp(self.Kernel))-0.4;
    return -tf.matmul(input, W)

layer = FCLayer_InhibitoryInput(10)

In [182]:
def myNetwork(input_shape = 4):
    # Define the input as a tensor with shape input_shape
    X_input = layers.Input(input_shape)
    
    # Inhibitory layer path
    X_inh = X_input
    X_inh = FCLayer_InhibitoryInput(4)(X_inh)
    
    # Add linear activations of X_input and X_inh
    R_linear = layers.Add()([X_inh,X_input])
    R = layers.Activation('relu')(R_linear)
    
    out = layers.Dense(1, activation='relu', name='C')(R) 
    
    model = Model(inputs = X_input, outputs = out, name='LatInhModel')

In [183]:
network = myNetwork(Xtrain.shape[1:])

network.compile(optimizer = "adam", loss = 'mean_squared_error', metrics = ["accuracy"])

AttributeError: in user code:

    <ipython-input-179-5e6d19f8e981>:13 call  *
        W = np.log(1+np.exp(self.Kernel))-0.4;

    AttributeError: 'FCLayer_InhibitoryInput' object has no attribute 'Kernel'
